In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
from torchvision.transforms import Compose, ToTensor
from torchsummary import summary
import pickle
import pandas as pd
import plotly.express as px
from tqdm import tqdm

In [3]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import accuracy_metrics

from torch.nn import CrossEntropyLoss
from torch.optim import SGD

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Libraries

In [4]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.training import MWUN
from base_code.plugins import WeightStoragePlugin

# Dataset and definitions

## Preprocessing definitions

In [5]:
train_transform = Compose(
    [
        ToTensor(),
    ]
)

test_transform = Compose(
    [
        ToTensor(),
    ]
)

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [6]:
# mnist_train = MNIST(DATASETS_PATH, train=True, download=True, transform=train_transform)
# mnist_test = MNIST(DATASETS_PATH, train=False, download=True, transform=test_transform)

scenario = PermutedMNIST(10, seed=1234, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [7]:
# scenario = nc_benchmark(
#     mnist_train, mnist_test,
#     n_experiences=len(mnist_train.classes), shuffle=True, seed=1234, task_labels=False
# )

train_stream = scenario.train_stream
test_stream = scenario.test_stream

## Evaluation metrics definition

In [8]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True), loggers=[InteractiveLogger()]
)

## Plugin defitinitions

In [9]:
model_plugins = [WeightStoragePlugin()]

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [11]:
# model = MLP(n_classes=scenario.n_classes, n_channels=1, width=28, height=28)
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100)
optimizer = SGD(model.parameters(), lr=1e-3)
criterion = CrossEntropyLoss()
strategy = MWUN(
    model,
    optimizer,
    criterion,
    eps=1e-5,
    train_epochs=5,
    train_mb_size=128,
    evaluator=eval_plugin,
    eval_mb_size=128,
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [12]:
results = list()

for experience in tqdm(train_stream):
    strategy.train(experience)
    metrics = strategy.eval(test_stream)
    results.append(metrics)

-- >> Start of training phase << --
100%|██████████| 469/469 [02:17<00:00,  3.41it/s]   
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 115.12it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:04<00:00, 114.58it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 114.26it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 115.74it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 145.77it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.7328
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 148.06it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.1734
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 141.62it/s]


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.2222
-- >> End of eval phase << --
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.2076
-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 113.04it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 115.60it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:04<00:00, 115.42it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 115.08it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 114.69it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 146.74it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.6734
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 143.90it/s]
> Eval on experience 1 (


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.2317
-- >> End of eval phase << --
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.2683
-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 114.38it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 114.08it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:04<00:00, 113.68it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 114.15it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 114.69it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 142.69it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.6337
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 146.27it/s]
> Eval on experience 1 (


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.1816
-- >> End of eval phase << --
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.3086
-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 113.35it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 114.64it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:04<00:00, 114.22it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 114.96it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 111.72it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 142.93it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.5654
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 146.90it/s]
> Eval on experience 1 (


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.1385
-- >> End of eval phase << --
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.3684
-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 114.69it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 114.86it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:04<00:00, 115.36it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 114.81it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 115.35it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 142.76it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.5458
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 146.93it/s]
> Eval on experience 1 (

...


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.1315
-- >> End of eval phase << --
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.5562
-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 114.35it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 114.67it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:04<00:00, 112.21it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 115.10it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 114.42it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 147.21it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.4745
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 141.68it/s]
> Eval on experience 1 (


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.1404
-- >> End of eval phase << --
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.6091
-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 114.34it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 114.74it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:04<00:00, 114.61it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 113.14it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 114.64it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 143.71it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.4542
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 144.06it/s]
> Eval on experience 1 (


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.1628
-- >> End of eval phase << --
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.6678
-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 114.66it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 114.83it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:04<00:00, 114.26it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 115.00it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 115.17it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 142.84it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.4533
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 146.72it/s]
> Eval on experience 1 (

100%|██████████| 10/10 [04:20<00:00, 26.03s/it]


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.8433
-- >> End of eval phase << --
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.7278


In [13]:
accuracies: dict[int, list[float]] = dict()

for i in range(10):
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp00{i}"
    ][1][-10:]

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1][-10:]

In [14]:
acc_df = pd.DataFrame(accuracies)
acc_df.index = range(10)

In [15]:
fig = px.line(acc_df, x=acc_df.index, y=acc_df.columns, range_y=[0, 1])
fig.show()

# Store metrics

In [15]:
pickle.dump(accuracies, open(SAVED_METRICS_PATH / "mwun.pkl", "wb"))